# Hybrid CNC Failure Diagnosis System - Interactive Demo

This notebook demonstrates the capabilities of the Hybrid CNC Failure Diagnosis System. 
It combines **Bayesian Networks (BN)** for probabilistic reasoning and **Knowledge Graphs (KG)** for contextual knowledge to diagnose failures and recommend maintenance actions.

In [ ]:
# === Setup ===
import os
import sys
import pandas as pd
import warnings

# Add src to path
sys.path.append(os.path.abspath(''))

from src.integration import run_real, evaluate_on_test_set
from src.bn_model import load_model, print_structure
from src.utils import load_cfg

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("Setup complete. Modules loaded.")

## 1. Load Configuration and Model
We load the system configuration and the pre-trained Bayesian Network model.

In [ ]:
cfg = load_cfg()
model_path = cfg['bn'].get('model_cache_path', 'models/trained_model.pkl')

if os.path.exists(model_path):
    model = load_model(model_path)
    print(f"\nModel loaded successfully from {model_path}")
else:
    print(f"Model not found at {model_path}. Please run main.py in 'real' mode to train it first.")

## 2. Visualize Bayesian Network Structure
Let's inspect the learned structure of the Bayesian Network, including the latent variables.

In [ ]:
if 'model' in locals():
    print_structure(model)

## 3. Live Diagnosis on Test Data
We will now pick random faulty samples from the test set and run the diagnosis pipeline.
This demonstrates the **Hybrid Reasoning** cycle:
1. **BN Inference**: Estimate failure probability and root causes.
2. **KG Query**: Retrieve maintenance procedures and costs.
3. **Decision Support**: Recommend the best action.

In [ ]:
# Load test data
_, test_data = run_real(None, debug=False, force_retrain=False, return_test_data=True)

# Filter for faulty samples (spindle_overheat=1)
faulty_samples = test_data[test_data["spindle_overheat"] == 1]

print(f"Found {len(faulty_samples)} faulty samples in test set.")

In [ ]:
# Pick a random sample and diagnose
if len(faulty_samples) > 0:
    sample = faulty_samples.sample(1).iloc[0]
    print(f"\n=== DIAGNOSING SAMPLE {sample.name} ===")
    
    # Extract evidence (only sensor columns)
    sensor_cols = cfg["bn"]["sensors"]
    evidence = {col: int(sample[col]) for col in sensor_cols if col in test_data.columns}
    print(f"Observed Sensors: {evidence}")
    
    # Run diagnosis
    run_real(evidence, debug=False, force_retrain=False)
else:
    print("No faulty samples available.")

## 4. System Evaluation
Finally, let's evaluate the model's performance on the entire test set.

In [ ]:
print("Running evaluation on full test set...")
results = evaluate_on_test_set(model, test_data, debug=False)